#CS344 Homework 2
##Myungha Kim
##March 8th, 2019
##Problem 1: Build a spam filter based on Paul Graham’s A Plan for Spam

##Problem 1: Build a spam filter based on Paul Graham’s A Plan for Spam

In [1]:
'''
This program takes in the sample text and test if it is a spam or not based on the data from
hash tables of words from spam corpus and non-spam corpus & calculated probability for each word being spam
'''


def Hash(spam_corpus, ham_corpus):
    spam = {}  # key as token; value as the number of occurence
    not_spam = {}
    token = []
    ngood = 0
    nbad = 0

    # 1st hash table for spam
    for msg in spam_corpus:
        nbad += 1
        for wd in msg:
            if wd in spam:
                spam[wd] += 1
            else:
                spam[wd] = 1
            if wd not in token:
                token.append(wd)

    # 2nd hash table for not_spam
    for msg in ham_corpus:
        ngood += 1
        for wd in msg:
            if wd in not_spam:
                not_spam[wd] += 1
            else:
                not_spam[wd] = 1
            if wd not in token:
                token.append(wd)

    # 3rd hash table - probability
    prob = {}
    for wd in token:
        if wd in not_spam:
            g = 2 * not_spam[wd]
        else:
            g = 0
        if wd in spam:
            b = spam[wd]
        else:
            b = 0
        if g + b > 1:
            prob[wd] = max(0.01, min(0.99, min(1.0, b/nbad) / (min(1.0, g/ngood) + min(1.0, b/nbad))))
        else:
            prob[wd] = 0
    print("Probability Hashtable:\n")
    return prob

def SpamCheck(text, filt):
    product = 1
    comp_product = 1

    for wd in text:
        if wd in filt:
            product *= filt[wd]
            comp_product *= 1 - filt[wd]
        elif wd not in filt:
            filt[wd] = 0.4
    marginal = product + comp_product
    Bayes = product / marginal
    if Bayes > 0.9:
        return True
    else:
        return False

# Hard-coded SPAM/HAM corpus

spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

sample_text = ["I", "am", "spam", "yes", "i", "am"]

spam_filter = Hash(spam_corpus, ham_corpus)
if SpamCheck(sample_text, spam_filter):
    print("Spam")
else:
    print("Not spam")

Probability Hashtable:

Spam


###Q. Graham argues that this is a Bayesian approach to SPAM. What makes it Bayesian?
Graham argues that this is a Bayesian approach to filter spam emails because the algorithm decides if the email is spam or not based on the given probability of each word in the email being spam or not. With the probability of each word (token) being spam or not, the probability of the email being spam or not via Bayesian statistical model which is the kernel / marginal at the end of the code.

##Problem 2: Do the following exercises based on the Bayesian network shown in Figure 14.12a
##Part a.

In [ ]:
from tools.aima.probability import BayesNet, enumeration_ask, elimination_ask

# Utility variables
T, F = True, False

# From AIMA code (probability.py) - Fig. 14_12 - Weather
weatherA = BayesNet([
    ('Cloudy', '', 0.5),
    ('Rain', 'Cloudy', {T: 0.80, F: 0.20}),
    ('Sprinkler', 'Cloudy', {T: 0.10, F: 0.50}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.90, (F, T): 0.90, (F, F): 0.00})
])


###Part b.
the number of independent values in the full joint probability distribution for this domain -> 2^4 = 16

Part c.
The number of independent values in the Bayesian network for this domain -> 9, counting connected bayesian networks shown in the figure

Part d.¶

In [ ]:
# Compute P(Cloudy)
print('P(Cloudy)')
print("Enumeration-ask -> " + enumeration_ask('Cloudy', dict(), weatherA).show_approx())

# Compute P(Sprinkler | cloudy)
print('\nP(Sprinkler | cloudy)')
print("Enumeration-ask -> " + enumeration_ask('Sprinkler', dict(Cloudy=T), weatherA).show_approx())

# Compute P(Cloudy| the sprinkler is running and it’s not raining)
print('\nP (Cloudy | sprinker and -rain')
print("Enumeration-ask -> " + enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), weatherA).show_approx())

# Compute P(WetGrass | it’s cloudy, the sprinkler is running and it’s raining)
print('\nP (WetGrass | cloudy and sprinkler and rain')
print("Enumeration-ask -> " + enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), weatherA).show_approx())

# Compute P(WetGrass | it’s cloudy, the sprinkler is running and it’s raining)
print('\nP (WetGrass | cloudy and sprinkler and rain')
print("Enumeration-ask -> " + enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), weatherA).show_approx())

# Compute P(Cloudy | the grass is not wet)
print('\nP (Cloudy | -wet grass)')
print("Enumeration-ask -> " + enumeration_ask('Cloudy', dict(WetGrass=F), weatherA).show_approx())

###Hand-worked derivations

###i. P(Cloudy) = <0.5, 0.5> ii. P(Sprinker | cloudy) = <0.1, 0.9> iii. P(Cloudy| sprinker ^ -rain) = alpha * <0.5 * 0.1 * 0.2, 0.5 * 0.5 * 0.8> = alpha * <0.01, 0.2> = <0.0476, 0.9524> iv. P(WetGrass | cloudy ^ sprinkler ^ rain) = alpha * <0.5 * 0.1 * 0.8 * 0.99, 0.5 * 0.1 * 0.8 * 0.01> = alpha * <0.99, 0.01> = <0.99, 0.01>
v. P(Cloudy|-wetgrass) = alpha * sum( P(Cloudy ^ sprinkler ^ rain ^ -wetgrass) ) = alpha * sum( P(Cloudy) * P(sprinkler ^ rain ^ -wetgrass | Cloudy) ) = alpha * <0.5 * 0.1 * 0.8 * 0.01 + 0.5 * 0.1 * 0.2 * 0.1 + 0.5 * 0.9 * 0.8 * 0.1 + 0.5 * 0.9 * 0.2 * 1.00, 0.5 * 0.5 * 0.2 * 0.01 + 0.5 * 0.5 * 0.8 * 0.1 + 0.5 * 0.5 * 0.2 * 0.1 + 0.5 * 0.5 * 0.8 * 1.00> = alpha * <0.1274, 0.2255> = <0.361, 0.639>